# 🔍 Mesin Pencari Data HP (Smartphone Query Engine)

## Apa Ini?

Ini adalah program sederhana untuk **mencari dan menganalisis data HP** menggunakan perintah teks.

Bayangkan seperti Google, tapi khusus untuk data HP. Kamu ketik perintah, program akan cari hasilnya!

---

## 🎯 Tujuan

| Tujuan | Penjelasan Singkat |
|--------|-------------------|
| 1. Belajar **Lexer** | Memecah kalimat jadi kata-kata |
| 2. Belajar **Parser** | Memahami struktur kalimat |
| 3. Belajar **Interpreter** | Menjalankan perintah |

---

## 🎁 Manfaat

- ✅ Bisa mencari HP berdasarkan merek, harga, RAM, dll
- ✅ Bisa mengubah teks (huruf besar/kecil)
- ✅ Bisa menghitung statistik (rata-rata harga, dll)
- ✅ Memahami cara kerja bahasa pemrograman

---

## 📱 Contoh Penggunaan

```
FILTER merek = 'Samsung'          → Cari HP Samsung
FILTER harga < 5000000            → Cari HP murah (dibawah 5 juta)
TRANSFORM uppercase(nama)         → Ubah nama jadi HURUF BESAR
STATS average(harga)              → Hitung rata-rata harga
```

---

**Dibuat oleh:** Rizal  
**Tanggal:** 16 Januari 2026

## Langkah 1: Persiapan

Pertama, kita siapkan dulu alat-alatnya (library Python).

In [1]:
# Install pandas menggunakan virtual environment
import subprocess
import sys

# Gunakan venv yang sudah dibuat
venv_pip = "/home/rizal/MyProject/Smartphone Query Engine/venv/bin/pip"
subprocess.check_call([venv_pip, "install", "pandas", "-q"])

# Tambahkan path venv ke sys.path
sys.path.insert(0, "/home/rizal/MyProject/Smartphone Query Engine/venv/lib/python3.14/site-packages")
print("✅ Setup selesai!")

✅ Setup selesai!


In [2]:
# Import library yang diperlukan
import pandas as pd
import re
from enum import Enum, auto
from typing import List, Tuple, Any, Dict, Optional
from dataclasses import dataclass

print("✅ Semua library berhasil di-import!")

✅ Semua library berhasil di-import!


## Langkah 2: Baca Data HP

Kita baca data HP dari file CSV. Data ini berisi info seperti nama HP, merek, harga, RAM, dll.

In [3]:
# Membaca data smartphone dari file CSV
df = pd.read_csv("data_smartphone.csv")

# Tampilkan informasi dasar tentang data
print("📱 DATA SMARTPHONE")
print("=" * 50)
print(f"Jumlah data: {len(df)} smartphone")
print(f"Kolom: {list(df.columns)}")
print("\n📋 Preview 5 data pertama:")
df.head()

📱 DATA SMARTPHONE
Jumlah data: 20 smartphone
Kolom: ['id', 'nama', 'merek', 'harga', 'ram', 'storage', 'baterai', 'kamera', 'layar', 'sistem_operasi', 'deskripsi']

📋 Preview 5 data pertama:


,id,nama,merek,harga,ram,storage,baterai,kamera,layar,sistem_operasi,deskripsi
0,1,Galaxy S24 Ultra,Samsung,21999000,12,256,5000,200,6.80,Android 14,Smartphone flagship dengan kamera 200MP dan S ...
1,2,iPhone 15 Pro Max,Apple,24999000,8,256,4422,48,6.70,iOS 17,Smartphone premium dengan chip A17 Pro dan Dyn...
2,3,Redmi Note 13 Pro,Xiaomi,3999000,8,128,5100,200,6.67,Android 13,HP mid-range dengan kamera 200MP dan fast char...
3,4,ROG Phone 8 Pro,ASUS,18999000,16,512,5500,50,6.78,Android 14,Gaming phone terbaik dengan layar 165Hz dan si...
4,5,Pixel 8 Pro,Google,15999000,12,128,5050,50,6.70,Android 14,Smartphone dengan AI terbaik dan pengalaman An...


In [4]:
# Tampilkan statistik data
print("📊 STATISTIK DATA")
print("=" * 50)
df.describe()

📊 STATISTIK DATA


,id,harga,ram,storage,baterai,kamera,layar
count,20.00000,2.000000e+01,20.000000,20.000000,20.000000,20.000000,20.000000
mean,10.50000,1.230400e+07,10.900000,275.200000,5042.550000,68.700000,6.691000
std,5.91608,6.847472e+06,3.275105,133.118704,604.901686,49.303678,0.175106
min,1.00000,1.799000e+06,4.000000,128.000000,3279.000000,12.000000,6.100000
25%,5.75000,6.999000e+06,8.000000,224.000000,5000.000000,50.000000,6.692500
50%,10.50000,1.349900e+07,12.000000,256.000000,5000.000000,50.000000,6.760000
75%,15.25000,1.724900e+07,12.000000,256.000000,5400.000000,50.000000,6.785000
max,20.00000,2.499900e+07,16.000000,512.000000,6500.000000,200.000000,6.820000


## Langkah 3: Buat LEXER (Pemecah Kalimat)

**Lexer** tugasnya memecah perintah jadi kata-kata kecil (token).

Contoh:
```
"FILTER merek = 'Samsung'"
     ↓ dipecah jadi ↓
[FILTER] [merek] [=] ['Samsung']
```

### Jenis-jenis Token:
| Jenis | Contoh |
|-------|--------|
| Kata Kunci | FILTER, TRANSFORM, STATS |
| Operator | =, <, >, != |
| Fungsi | uppercase, count, average |
| Teks | "Samsung", "Xiaomi" |
| Angka | 5000000, 8 |
| Nama Kolom | nama, merek, harga |

In [5]:
# ============================================
# LEXER - Tokenizer untuk Query Engine
# ============================================

class TokenType(Enum):
    """Tipe-tipe token yang dikenali oleh lexer"""
    # Kata kunci utama
    KEYWORD = auto()      # FILTER, TRANSFORM, STATS, SELECT, WHERE, AND, OR
    
    # Operator perbandingan dan logika
    OPERATOR = auto()     # =, <, >, <=, >=, !=, CONTAINS, LIKE
    
    # Fungsi transformasi dan statistik
    FUNCTION = auto()     # uppercase, lowercase, count, sum, average, min, max
    
    # Literal values
    STRING = auto()       # "Samsung", 'Xiaomi'
    NUMBER = auto()       # 5000000, 8.5
    
    # Identifier (nama kolom)
    IDENTIFIER = auto()   # nama, merek, harga
    
    # Simbol
    LPAREN = auto()       # (
    RPAREN = auto()       # )
    COMMA = auto()        # ,
    
    # Akhir input
    EOF = auto()


@dataclass
class Token:
    """Struktur data untuk menyimpan informasi token"""
    type: TokenType
    value: Any
    position: int  # Posisi token dalam string query
    
    def __repr__(self):
        return f"Token({self.type.name}, '{self.value}', pos={self.position})"


class Lexer:
    """
    Lexer (Tokenizer) untuk memecah query menjadi token-token.
    
    Contoh:
        Input:  "FILTER merek = 'Samsung'"
        Output: [Token(KEYWORD, 'FILTER'), Token(IDENTIFIER, 'merek'), 
                 Token(OPERATOR, '='), Token(STRING, 'Samsung')]
    """
    
    # Kata kunci yang dikenali
    KEYWORDS = {'FILTER', 'TRANSFORM', 'STATS', 'SELECT', 'WHERE', 'AND', 'OR', 'NOT'}
    
    # Fungsi yang didukung
    FUNCTIONS = {
        # Transformasi teks
        'uppercase', 'lowercase', 'capitalize', 'trim', 'length',
        # Statistik
        'count', 'sum', 'average', 'min', 'max', 'distinct'
    }
    
    # Operator yang didukung
    OPERATORS = {'=', '<', '>', '<=', '>=', '!=', 'CONTAINS', 'LIKE'}
    
    def __init__(self, query: str):
        """Inisialisasi lexer dengan query string"""
        self.query = query
        self.pos = 0
        self.current_char = self.query[0] if query else None
    
    def error(self, message: str):
        """Raise exception dengan pesan error"""
        raise SyntaxError(f"Lexer Error di posisi {self.pos}: {message}")
    
    def advance(self):
        """Maju ke karakter berikutnya"""
        self.pos += 1
        if self.pos < len(self.query):
            self.current_char = self.query[self.pos]
        else:
            self.current_char = None
    
    def skip_whitespace(self):
        """Lewati spasi dan whitespace"""
        while self.current_char and self.current_char.isspace():
            self.advance()
    
    def read_number(self) -> Token:
        """Baca token angka (integer atau float)"""
        start_pos = self.pos
        result = ''
        
        while self.current_char and (self.current_char.isdigit() or self.current_char == '.'):
            result += self.current_char
            self.advance()
        
        # Konversi ke int atau float
        if '.' in result:
            return Token(TokenType.NUMBER, float(result), start_pos)
        return Token(TokenType.NUMBER, int(result), start_pos)
    
    def read_string(self) -> Token:
        """Baca token string (dalam tanda kutip)"""
        start_pos = self.pos
        quote_char = self.current_char  # ' atau "
        self.advance()  # Skip opening quote
        
        result = ''
        while self.current_char and self.current_char != quote_char:
            result += self.current_char
            self.advance()
        
        if self.current_char != quote_char:
            self.error(f"String tidak ditutup dengan {quote_char}")
        
        self.advance()  # Skip closing quote
        return Token(TokenType.STRING, result, start_pos)
    
    def read_identifier(self) -> Token:
        """Baca identifier atau keyword"""
        start_pos = self.pos
        result = ''
        
        while self.current_char and (self.current_char.isalnum() or self.current_char == '_'):
            result += self.current_char
            self.advance()
        
        upper_result = result.upper()
        
        # Cek apakah ini keyword
        if upper_result in self.KEYWORDS:
            return Token(TokenType.KEYWORD, upper_result, start_pos)
        
        # Cek apakah ini operator teks
        if upper_result in ['CONTAINS', 'LIKE']:
            return Token(TokenType.OPERATOR, upper_result, start_pos)
        
        # Cek apakah ini function
        if result.lower() in self.FUNCTIONS:
            return Token(TokenType.FUNCTION, result.lower(), start_pos)
        
        # Jika tidak, ini adalah identifier (nama kolom)
        return Token(TokenType.IDENTIFIER, result, start_pos)
    
    def read_operator(self) -> Token:
        """Baca operator perbandingan"""
        start_pos = self.pos
        result = self.current_char
        self.advance()
        
        # Cek operator 2 karakter (<=, >=, !=)
        if self.current_char == '=' and result in '<>!':
            result += self.current_char
            self.advance()
        
        if result not in self.OPERATORS:
            self.error(f"Operator tidak dikenali: {result}")
        
        return Token(TokenType.OPERATOR, result, start_pos)
    
    def tokenize(self) -> List[Token]:
        """
        Proses utama tokenisasi - memecah query menjadi list token
        
        Returns:
            List[Token]: Daftar token dari query
        """
        tokens = []
        
        while self.current_char:
            # Skip whitespace
            if self.current_char.isspace():
                self.skip_whitespace()
                continue
            
            # Number
            if self.current_char.isdigit():
                tokens.append(self.read_number())
                continue
            
            # String (dalam kutip)
            if self.current_char in '"\'':
                tokens.append(self.read_string())
                continue
            
            # Identifier/Keyword/Function
            if self.current_char.isalpha() or self.current_char == '_':
                tokens.append(self.read_identifier())
                continue
            
            # Operator
            if self.current_char in '=<>!':
                tokens.append(self.read_operator())
                continue
            
            # Simbol khusus
            if self.current_char == '(':
                tokens.append(Token(TokenType.LPAREN, '(', self.pos))
                self.advance()
                continue
            
            if self.current_char == ')':
                tokens.append(Token(TokenType.RPAREN, ')', self.pos))
                self.advance()
                continue
            
            if self.current_char == ',':
                tokens.append(Token(TokenType.COMMA, ',', self.pos))
                self.advance()
                continue
            
            # Karakter tidak dikenali
            self.error(f"Karakter tidak dikenali: '{self.current_char}'")
        
        # Tambahkan token EOF
        tokens.append(Token(TokenType.EOF, None, self.pos))
        return tokens


print("✅ Lexer berhasil dibuat!")
print("\n📖 Cara kerja Lexer:")
print("   Query String → [Token1, Token2, Token3, ...]")

✅ Lexer berhasil dibuat!

📖 Cara kerja Lexer:
   Query String → [Token1, Token2, Token3, ...]


In [6]:
# 🧪 TEST LEXER
print("🧪 TEST LEXER")
print("=" * 60)

# Contoh query untuk di-tokenize
test_queries = [
    "FILTER merek = 'Samsung'",
    "FILTER harga < 5000000",
    "TRANSFORM uppercase(nama)",
    "STATS count(nama), average(harga)"
]

for query in test_queries:
    print(f"\n📝 Query: {query}")
    lexer = Lexer(query)
    tokens = lexer.tokenize()
    print(f"   Tokens: {tokens[:-1]}")  # Exclude EOF token

🧪 TEST LEXER

📝 Query: FILTER merek = 'Samsung'
   Tokens: [Token(KEYWORD, 'FILTER', pos=0), Token(IDENTIFIER, 'merek', pos=7), Token(OPERATOR, '=', pos=13), Token(STRING, 'Samsung', pos=15)]

📝 Query: FILTER harga < 5000000
   Tokens: [Token(KEYWORD, 'FILTER', pos=0), Token(IDENTIFIER, 'harga', pos=7), Token(OPERATOR, '<', pos=13), Token(NUMBER, '5000000', pos=15)]

📝 Query: TRANSFORM uppercase(nama)
   Tokens: [Token(KEYWORD, 'TRANSFORM', pos=0), Token(FUNCTION, 'uppercase', pos=10), Token(LPAREN, '(', pos=19), Token(IDENTIFIER, 'nama', pos=20), Token(RPAREN, ')', pos=24)]

📝 Query: STATS count(nama), average(harga)
   Tokens: [Token(KEYWORD, 'STATS', pos=0), Token(FUNCTION, 'count', pos=6), Token(LPAREN, '(', pos=11), Token(IDENTIFIER, 'nama', pos=12), Token(RPAREN, ')', pos=16), Token(COMMA, ',', pos=17), Token(FUNCTION, 'average', pos=19), Token(LPAREN, '(', pos=26), Token(IDENTIFIER, 'harga', pos=27), Token(RPAREN, ')', pos=32)]


## Langkah 4: Buat PARSER dengan Automata

**Parser** tugasnya memahami struktur kalimat. Pakai **Automata** (mesin status) untuk cek apakah perintah sudah benar.

### Cara Kerja Sederhana:

```
MULAI → Baca FILTER/TRANSFORM/STATS → Proses kondisi → SELESAI ✅
```

### Diagram:
```
         [MULAI]
            │
    ┌───────┼───────┐
    │       │       │
 FILTER  TRANSFORM STATS
    │       │       │
    ▼       ▼       ▼
 [Cek]   [Fungsi] [Hitung]
    │       │       │
    ▼       ▼       ▼
       [SELESAI ✅]
```

Kalau perintah salah, automata akan kasih error!

In [7]:
# ============================================
# AUTOMATA - Simulasi Finite State Machine
# ============================================

class AutomataState(Enum):
    """State dalam automata untuk parsing query"""
    START = auto()           # State awal
    FILTER_STATE = auto()    # Setelah FILTER keyword
    TRANSFORM_STATE = auto() # Setelah TRANSFORM keyword
    STATS_STATE = auto()     # Setelah STATS keyword
    SELECT_STATE = auto()    # Setelah SELECT keyword
    CONDITION = auto()       # Membaca kondisi filter
    FUNCTION_CALL = auto()   # Membaca pemanggilan fungsi
    ACCEPT = auto()          # State akhir (query valid)
    ERROR = auto()           # State error


class QueryAutomata:
    """
    Simulasi Finite State Automata untuk validasi query.
    
    Automata ini digunakan untuk:
    1. Validasi struktur sintaks query
    2. Menentukan tipe query (FILTER/TRANSFORM/STATS)
    3. Melacak transisi state selama parsing
    """
    
    # Tabel transisi state
    # Format: {(current_state, input_type): next_state}
    TRANSITIONS = {
        # Dari START, bisa ke berbagai command
        (AutomataState.START, 'FILTER'): AutomataState.FILTER_STATE,
        (AutomataState.START, 'TRANSFORM'): AutomataState.TRANSFORM_STATE,
        (AutomataState.START, 'STATS'): AutomataState.STATS_STATE,
        (AutomataState.START, 'SELECT'): AutomataState.SELECT_STATE,
        
        # Dari FILTER_STATE, baca kondisi
        (AutomataState.FILTER_STATE, 'IDENTIFIER'): AutomataState.CONDITION,
        
        # Dari CONDITION, bisa selesai atau lanjut AND/OR
        (AutomataState.CONDITION, 'AND'): AutomataState.FILTER_STATE,
        (AutomataState.CONDITION, 'OR'): AutomataState.FILTER_STATE,
        (AutomataState.CONDITION, 'EOF'): AutomataState.ACCEPT,
        
        # Dari TRANSFORM_STATE, baca function
        (AutomataState.TRANSFORM_STATE, 'FUNCTION'): AutomataState.FUNCTION_CALL,
        
        # Dari FUNCTION_CALL, selesai
        (AutomataState.FUNCTION_CALL, 'EOF'): AutomataState.ACCEPT,
        (AutomataState.FUNCTION_CALL, 'COMMA'): AutomataState.STATS_STATE,
        
        # Dari STATS_STATE, baca stat function
        (AutomataState.STATS_STATE, 'FUNCTION'): AutomataState.FUNCTION_CALL,
    }
    
    def __init__(self):
        """Inisialisasi automata di state START"""
        self.current_state = AutomataState.START
        self.history = [AutomataState.START]  # Riwayat transisi
    
    def transition(self, input_symbol: str) -> bool:
        """
        Lakukan transisi berdasarkan input symbol.
        
        Args:
            input_symbol: Symbol input (keyword, token type, dll)
            
        Returns:
            True jika transisi valid, False jika tidak
        """
        key = (self.current_state, input_symbol)
        
        if key in self.TRANSITIONS:
            self.current_state = self.TRANSITIONS[key]
            self.history.append(self.current_state)
            return True
        
        # Transisi tidak valid
        self.current_state = AutomataState.ERROR
        self.history.append(AutomataState.ERROR)
        return False
    
    def is_accepting(self) -> bool:
        """Cek apakah automata di state accepting"""
        return self.current_state == AutomataState.ACCEPT
    
    def is_error(self) -> bool:
        """Cek apakah automata di state error"""
        return self.current_state == AutomataState.ERROR
    
    def get_state_name(self) -> str:
        """Dapatkan nama state saat ini"""
        return self.current_state.name
    
    def print_history(self):
        """Cetak riwayat transisi state"""
        print("📍 Riwayat Transisi State:")
        for i, state in enumerate(self.history):
            arrow = " → " if i < len(self.history) - 1 else ""
            print(f"   {state.name}{arrow}", end="")
        print()


print("✅ Automata berhasil dibuat!")
print("\n📖 Cara kerja Automata:")
print("   START → [keyword] → [parse state] → ... → ACCEPT/ERROR")

✅ Automata berhasil dibuat!

📖 Cara kerja Automata:
   START → [keyword] → [parse state] → ... → ACCEPT/ERROR


In [8]:
# ============================================
# PARSER - Menganalisis struktur query
# ============================================

@dataclass
class ASTNode:
    """Node dalam Abstract Syntax Tree (AST)"""
    type: str           # Tipe node: 'filter', 'transform', 'stats', dll
    value: Any = None   # Nilai node
    children: List = None  # Child nodes
    
    def __post_init__(self):
        if self.children is None:
            self.children = []
    
    def __repr__(self):
        return f"ASTNode({self.type}, {self.value}, children={len(self.children)})"


class Parser:
    """
    Parser untuk mengubah token menjadi Abstract Syntax Tree (AST).
    
    Parser ini menggunakan metode Recursive Descent dengan bantuan
    automata untuk validasi struktur.
    """
    
    def __init__(self, tokens: List[Token]):
        """Inisialisasi parser dengan list token"""
        self.tokens = tokens
        self.pos = 0
        self.current_token = tokens[0] if tokens else None
        self.automata = QueryAutomata()  # Automata untuk validasi
    
    def error(self, message: str):
        """Raise exception dengan pesan error"""
        raise SyntaxError(f"Parser Error di token {self.pos}: {message}")
    
    def advance(self):
        """Maju ke token berikutnya"""
        self.pos += 1
        if self.pos < len(self.tokens):
            self.current_token = self.tokens[self.pos]
        else:
            self.current_token = Token(TokenType.EOF, None, -1)
    
    def expect(self, token_type: TokenType, value: Any = None):
        """Expect token tertentu, error jika tidak cocok"""
        if self.current_token.type != token_type:
            self.error(f"Expected {token_type.name}, got {self.current_token.type.name}")
        if value and self.current_token.value != value:
            self.error(f"Expected '{value}', got '{self.current_token.value}'")
        token = self.current_token
        self.advance()
        return token
    
    def parse(self) -> ASTNode:
        """
        Parsing utama - mengubah tokens menjadi AST.
        
        Returns:
            ASTNode: Root node dari AST
        """
        if self.current_token.type == TokenType.EOF:
            self.error("Query kosong")
        
        # Cek keyword utama
        if self.current_token.type != TokenType.KEYWORD:
            self.error(f"Query harus dimulai dengan keyword (FILTER/TRANSFORM/STATS)")
        
        keyword = self.current_token.value
        
        # Transisi automata ke state yang sesuai
        if not self.automata.transition(keyword):
            self.error(f"Keyword tidak valid: {keyword}")
        
        # Parse berdasarkan keyword
        if keyword == 'FILTER':
            return self.parse_filter()
        elif keyword == 'TRANSFORM':
            return self.parse_transform()
        elif keyword == 'STATS':
            return self.parse_stats()
        elif keyword == 'SELECT':
            return self.parse_select()
        else:
            self.error(f"Keyword tidak didukung: {keyword}")
    
    def parse_filter(self) -> ASTNode:
        """Parse FILTER command"""
        self.advance()  # Skip 'FILTER'
        
        root = ASTNode('filter')
        
        # Parse kondisi pertama
        condition = self.parse_condition()
        root.children.append(condition)
        
        # Parse kondisi tambahan (AND/OR)
        while self.current_token.type == TokenType.KEYWORD and \
              self.current_token.value in ('AND', 'OR'):
            
            logical_op = self.current_token.value
            self.automata.transition(logical_op)
            self.advance()
            
            condition = self.parse_condition()
            condition.value = {'operator': logical_op, 'condition': condition.value}
            root.children.append(condition)
        
        # Transisi ke ACCEPT
        self.automata.transition('EOF')
        
        return root
    
    def parse_condition(self) -> ASTNode:
        """Parse kondisi: identifier operator value"""
        # Transisi automata
        self.automata.transition('IDENTIFIER')
        
        # Parse identifier (nama kolom)
        if self.current_token.type != TokenType.IDENTIFIER:
            self.error(f"Expected identifier, got {self.current_token.type.name}")
        
        column = self.current_token.value
        self.advance()
        
        # Parse operator
        if self.current_token.type != TokenType.OPERATOR:
            self.error(f"Expected operator, got {self.current_token.type.name}")
        
        operator = self.current_token.value
        self.advance()
        
        # Parse value
        if self.current_token.type not in (TokenType.STRING, TokenType.NUMBER, TokenType.IDENTIFIER):
            self.error(f"Expected value, got {self.current_token.type.name}")
        
        value = self.current_token.value
        self.advance()
        
        return ASTNode('condition', {'column': column, 'operator': operator, 'value': value})
    
    def parse_transform(self) -> ASTNode:
        """Parse TRANSFORM command"""
        self.advance()  # Skip 'TRANSFORM'
        
        root = ASTNode('transform')
        
        # Parse function call(s)
        while self.current_token.type == TokenType.FUNCTION:
            func_node = self.parse_function_call()
            root.children.append(func_node)
            
            # Cek jika ada koma untuk fungsi berikutnya
            if self.current_token.type == TokenType.COMMA:
                self.advance()
            else:
                break
        
        # Transisi ke ACCEPT
        self.automata.transition('EOF')
        
        return root
    
    def parse_stats(self) -> ASTNode:
        """Parse STATS command"""
        self.advance()  # Skip 'STATS'
        
        root = ASTNode('stats')
        
        # Parse stat function(s)
        while self.current_token.type == TokenType.FUNCTION:
            func_node = self.parse_function_call()
            root.children.append(func_node)
            
            # Cek jika ada koma untuk fungsi berikutnya
            if self.current_token.type == TokenType.COMMA:
                self.automata.transition('COMMA')
                self.advance()
            else:
                break
        
        # Transisi ke ACCEPT
        self.automata.transition('EOF')
        
        return root
    
    def parse_select(self) -> ASTNode:
        """Parse SELECT command"""
        self.advance()  # Skip 'SELECT'
        
        root = ASTNode('select')
        
        # Parse column list
        while self.current_token.type == TokenType.IDENTIFIER:
            col_node = ASTNode('column', self.current_token.value)
            root.children.append(col_node)
            self.advance()
            
            if self.current_token.type == TokenType.COMMA:
                self.advance()
            else:
                break
        
        # Cek WHERE clause
        if self.current_token.type == TokenType.KEYWORD and \
           self.current_token.value == 'WHERE':
            self.advance()
            condition = self.parse_condition()
            root.children.append(ASTNode('where', children=[condition]))
        
        return root
    
    def parse_function_call(self) -> ASTNode:
        """Parse function call: func_name(column)"""
        # Transisi automata
        self.automata.transition('FUNCTION')
        
        func_name = self.current_token.value
        self.advance()
        
        # Expect (
        self.expect(TokenType.LPAREN)
        
        # Parse argument (column name)
        if self.current_token.type != TokenType.IDENTIFIER:
            self.error(f"Expected column name, got {self.current_token.type.name}")
        
        column = self.current_token.value
        self.advance()
        
        # Expect )
        self.expect(TokenType.RPAREN)
        
        return ASTNode('function', {'name': func_name, 'column': column})


print("✅ Parser berhasil dibuat!")
print("\n📖 Cara kerja Parser:")
print("   Tokens → AST (Abstract Syntax Tree)")

✅ Parser berhasil dibuat!

📖 Cara kerja Parser:
   Tokens → AST (Abstract Syntax Tree)


In [9]:
# 🧪 TEST PARSER
print("🧪 TEST PARSER")
print("=" * 60)

test_query = "FILTER merek = 'Samsung' AND harga < 10000000"
print(f"\n📝 Query: {test_query}")

# Tokenize
lexer = Lexer(test_query)
tokens = lexer.tokenize()
print(f"\n🔤 Tokens: {tokens[:-1]}")

# Parse
parser = Parser(tokens)
ast = parser.parse()
print(f"\n🌳 AST Root: {ast}")
print(f"   Children: {ast.children}")

# Tampilkan riwayat automata
parser.automata.print_history()

🧪 TEST PARSER

📝 Query: FILTER merek = 'Samsung' AND harga < 10000000

🔤 Tokens: [Token(KEYWORD, 'FILTER', pos=0), Token(IDENTIFIER, 'merek', pos=7), Token(OPERATOR, '=', pos=13), Token(STRING, 'Samsung', pos=15), Token(KEYWORD, 'AND', pos=25), Token(IDENTIFIER, 'harga', pos=29), Token(OPERATOR, '<', pos=35), Token(NUMBER, '10000000', pos=37)]

🌳 AST Root: ASTNode(filter, None, children=2)
   Children: [ASTNode(condition, {'column': 'merek', 'operator': '=', 'value': 'Samsung'}, children=0), ASTNode(condition, {'operator': 'AND', 'condition': {'column': 'harga', 'operator': '<', 'value': 10000000}}, children=0)]
📍 Riwayat Transisi State:
   START →    FILTER_STATE →    CONDITION →    FILTER_STATE →    CONDITION →    ACCEPT


## Langkah 5: Buat INTERPRETER (Penerjemah)

**Interpreter** tugasnya menjalankan perintah dan menghasilkan data.

### Fungsi yang Bisa Dipakai:

| Jenis | Fungsi | Kegunaan |
|-------|--------|----------|
| **Filter** | =, <, >, !=, CONTAINS | Cari data |
| **Transform** | uppercase, lowercase | Ubah teks |
| **Stats** | count, average, min, max | Hitung statistik |

In [10]:
# ============================================
# INTERPRETER - Menjalankan Query pada Data
# ============================================

class Interpreter:
    """
    Interpreter untuk menjalankan AST query pada DataFrame.
    
    Interpreter ini mengeksekusi query dan menghasilkan:
    - DataFrame hasil filter
    - DataFrame hasil transformasi
    - Dictionary hasil statistik
    """
    
    def __init__(self, df: pd.DataFrame):
        """Inisialisasi interpreter dengan DataFrame"""
        self.df = df.copy()
        self.result = None
    
    def execute(self, ast: ASTNode) -> Any:
        """
        Eksekusi AST dan kembalikan hasilnya.
        
        Args:
            ast: Abstract Syntax Tree dari parser
            
        Returns:
            DataFrame atau dictionary hasil eksekusi
        """
        if ast.type == 'filter':
            return self.execute_filter(ast)
        elif ast.type == 'transform':
            return self.execute_transform(ast)
        elif ast.type == 'stats':
            return self.execute_stats(ast)
        elif ast.type == 'select':
            return self.execute_select(ast)
        else:
            raise ValueError(f"Unknown AST type: {ast.type}")
    
    def execute_filter(self, ast: ASTNode) -> pd.DataFrame:
        """Eksekusi FILTER command"""
        result_df = self.df.copy()
        final_mask = None
        
        for i, child in enumerate(ast.children):
            condition = child.value
            
            # Cek apakah kondisi ini punya operator AND/OR (kondisi kedua dst)
            if isinstance(condition, dict) and 'operator' in condition and 'condition' in condition:
                logical_op = condition['operator']
                cond_data = condition['condition']
            else:
                logical_op = None
                cond_data = condition
            
            # Pastikan cond_data adalah dictionary dengan keys yang benar
            if not isinstance(cond_data, dict) or 'column' not in cond_data:
                raise ValueError(f"Format kondisi tidak valid: {cond_data}")
            
            # Bangun mask untuk kondisi
            column = cond_data['column']
            operator = cond_data['operator']
            value = cond_data['value']
            
            mask = self.evaluate_condition(result_df, column, operator, value)
            
            # Apply logical operator
            if i == 0 or final_mask is None:
                final_mask = mask
            elif logical_op == 'AND':
                final_mask = final_mask & mask
            elif logical_op == 'OR':
                final_mask = final_mask | mask
            else:
                final_mask = final_mask & mask
        
        return result_df[final_mask]
    
    def evaluate_condition(self, df: pd.DataFrame, column: str, operator: str, value: Any) -> pd.Series:
        """Evaluasi satu kondisi filter"""
        if column not in df.columns:
            raise ValueError(f"Kolom '{column}' tidak ditemukan dalam data")
        
        col = df[column]
        
        # Operator perbandingan standar
        if operator == '=':
            return col == value
        elif operator == '!=':
            return col != value
        elif operator == '<':
            return col < value
        elif operator == '>':
            return col > value
        elif operator == '<=':
            return col <= value
        elif operator == '>=':
            return col >= value
        elif operator == 'CONTAINS':
            return col.astype(str).str.contains(str(value), case=False, na=False)
        elif operator == 'LIKE':
            # Konversi SQL LIKE pattern ke regex
            pattern = str(value).replace('%', '.*').replace('_', '.')
            return col.astype(str).str.match(pattern, case=False, na=False)
        else:
            raise ValueError(f"Operator tidak dikenali: {operator}")
    
    def execute_transform(self, ast: ASTNode) -> pd.DataFrame:
        """Eksekusi TRANSFORM command"""
        result_df = self.df.copy()
        
        for child in ast.children:
            func_name = child.value['name']
            column = child.value['column']
            
            if column not in result_df.columns:
                raise ValueError(f"Kolom '{column}' tidak ditemukan")
            
            # Apply transformasi
            new_col_name = f"{func_name}_{column}"
            result_df[new_col_name] = self.apply_transform(result_df[column], func_name)
        
        return result_df
    
    def apply_transform(self, series: pd.Series, func_name: str) -> pd.Series:
        """Apply fungsi transformasi pada series"""
        str_series = series.astype(str)
        
        if func_name == 'uppercase':
            return str_series.str.upper()
        elif func_name == 'lowercase':
            return str_series.str.lower()
        elif func_name == 'capitalize':
            return str_series.str.title()
        elif func_name == 'trim':
            return str_series.str.strip()
        elif func_name == 'length':
            return str_series.str.len()
        else:
            raise ValueError(f"Fungsi transformasi tidak dikenali: {func_name}")
    
    def execute_stats(self, ast: ASTNode) -> Dict[str, Any]:
        """Eksekusi STATS command"""
        results = {}
        
        for child in ast.children:
            func_name = child.value['name']
            column = child.value['column']
            
            if column not in self.df.columns:
                raise ValueError(f"Kolom '{column}' tidak ditemukan")
            
            # Hitung statistik
            stat_key = f"{func_name}({column})"
            results[stat_key] = self.calculate_stat(self.df[column], func_name)
        
        return results
    
    def calculate_stat(self, series: pd.Series, func_name: str) -> Any:
        """Hitung statistik untuk series"""
        if func_name == 'count':
            return len(series)
        elif func_name == 'sum':
            return series.sum()
        elif func_name == 'average':
            return series.mean()
        elif func_name == 'min':
            return series.min()
        elif func_name == 'max':
            return series.max()
        elif func_name == 'distinct':
            return series.nunique()
        else:
            raise ValueError(f"Fungsi statistik tidak dikenali: {func_name}")
    
    def execute_select(self, ast: ASTNode) -> pd.DataFrame:
        """Eksekusi SELECT command"""
        columns = []
        where_condition = None
        
        for child in ast.children:
            if child.type == 'column':
                columns.append(child.value)
            elif child.type == 'where':
                where_condition = child.children[0]
        
        result_df = self.df.copy()
        
        # Apply WHERE jika ada
        if where_condition:
            cond = where_condition.value
            mask = self.evaluate_condition(result_df, cond['column'], cond['operator'], cond['value'])
            result_df = result_df[mask]
        
        # Select columns
        if columns:
            missing = [c for c in columns if c not in result_df.columns]
            if missing:
                raise ValueError(f"Kolom tidak ditemukan: {missing}")
            result_df = result_df[columns]
        
        return result_df


print("✅ Interpreter berhasil dibuat!")
print("\n📖 Cara kerja Interpreter:")
print("   AST + DataFrame → Hasil Query (DataFrame/Dict)")

✅ Interpreter berhasil dibuat!

📖 Cara kerja Interpreter:
   AST + DataFrame → Hasil Query (DataFrame/Dict)


## Langkah 6: Gabungkan Semua Jadi Query Engine

In [11]:
# ============================================
# QUERY ENGINE - Menggabungkan Semua Komponen
# ============================================

class SmartphoneQueryEngine:
    """
    Mini Query Engine untuk Data Smartphone.
    
    Query engine ini menggabungkan:
    1. Lexer - tokenisasi query
    2. Parser - analisis sintaks dengan automata
    3. Interpreter - eksekusi query
    
    Contoh penggunaan:
        engine = SmartphoneQueryEngine(df)
        result = engine.query("FILTER merek = 'Samsung'")
    """
    
    def __init__(self, df: pd.DataFrame):
        """Inisialisasi query engine dengan DataFrame"""
        self.df = df
        self.last_tokens = None
        self.last_ast = None
        self.last_automata = None
    
    def query(self, query_string: str, verbose: bool = False) -> Any:
        """
        Eksekusi query dan kembalikan hasilnya.
        
        Args:
            query_string: String query dalam bahasa query kita
            verbose: Jika True, tampilkan detail proses
            
        Returns:
            DataFrame atau Dictionary hasil query
        """
        if verbose:
            print("=" * 60)
            print(f"🔍 QUERY: {query_string}")
            print("=" * 60)
        
        try:
            # Step 1: Lexical Analysis (Tokenisasi)
            if verbose:
                print("\n📌 STEP 1: Lexical Analysis (Tokenisasi)")
            
            lexer = Lexer(query_string)
            tokens = lexer.tokenize()
            self.last_tokens = tokens
            
            if verbose:
                print(f"   Tokens: {[str(t) for t in tokens[:-1]]}")
            
            # Step 2: Syntax Analysis (Parsing dengan Automata)
            if verbose:
                print("\n📌 STEP 2: Syntax Analysis (Parsing)")
            
            parser = Parser(tokens)
            ast = parser.parse()
            self.last_ast = ast
            self.last_automata = parser.automata
            
            if verbose:
                print(f"   AST Root: {ast.type}")
                print(f"   Children: {len(ast.children)} node(s)")
                parser.automata.print_history()
            
            # Step 3: Execution (Interpretasi)
            if verbose:
                print("\n📌 STEP 3: Execution (Interpretasi)")
            
            interpreter = Interpreter(self.df)
            result = interpreter.execute(ast)
            
            if verbose:
                if isinstance(result, pd.DataFrame):
                    print(f"   Hasil: DataFrame dengan {len(result)} baris")
                else:
                    print(f"   Hasil: {result}")
            
            return result
            
        except SyntaxError as e:
            print(f"❌ SYNTAX ERROR: {e}")
            return None
        except ValueError as e:
            print(f"❌ VALUE ERROR: {e}")
            return None
        except Exception as e:
            print(f"❌ ERROR: {e}")
            return None
    
    def help(self):
        """Tampilkan bantuan penggunaan query engine"""
        help_text = """
╔══════════════════════════════════════════════════════════════╗
║           📖 PANDUAN SMARTPHONE QUERY ENGINE                 ║
╠══════════════════════════════════════════════════════════════╣
║                                                              ║
║  🔸 FILTER - Menyaring data                                  ║
║     FILTER kolom = 'nilai'                                   ║
║     FILTER kolom < angka                                     ║
║     FILTER kolom CONTAINS 'teks'                             ║
║     FILTER kondisi1 AND kondisi2                             ║
║     FILTER kondisi1 OR kondisi2                              ║
║                                                              ║
║  🔸 TRANSFORM - Mengubah format teks                         ║
║     TRANSFORM uppercase(kolom)                               ║
║     TRANSFORM lowercase(kolom)                               ║
║     TRANSFORM capitalize(kolom)                              ║
║     TRANSFORM length(kolom)                                  ║
║                                                              ║
║  🔸 STATS - Analisis statistik                               ║
║     STATS count(kolom)                                       ║
║     STATS sum(kolom), average(kolom)                         ║
║     STATS min(kolom), max(kolom)                             ║
║     STATS distinct(kolom)                                    ║
║                                                              ║
║  🔸 SELECT - Memilih kolom tertentu                          ║
║     SELECT kolom1, kolom2                                    ║
║     SELECT kolom1 WHERE kondisi                              ║
║                                                              ║
╚══════════════════════════════════════════════════════════════╝
        """
        print(help_text)


# Buat instance query engine
engine = SmartphoneQueryEngine(df)

print("✅ Query Engine berhasil dibuat!")
print("\n💡 Gunakan engine.query('QUERY_STRING') untuk menjalankan query")
print("💡 Gunakan engine.help() untuk melihat panduan")

✅ Query Engine berhasil dibuat!

💡 Gunakan engine.query('QUERY_STRING') untuk menjalankan query
💡 Gunakan engine.help() untuk melihat panduan


In [12]:
# Tampilkan panduan penggunaan
engine.help()


╔══════════════════════════════════════════════════════════════╗
║           📖 PANDUAN SMARTPHONE QUERY ENGINE                 ║
╠══════════════════════════════════════════════════════════════╣
║                                                              ║
║  🔸 FILTER - Menyaring data                                  ║
║     FILTER kolom = 'nilai'                                   ║
║     FILTER kolom < angka                                     ║
║     FILTER kolom CONTAINS 'teks'                             ║
║     FILTER kondisi1 AND kondisi2                             ║
║     FILTER kondisi1 OR kondisi2                              ║
║                                                              ║
║  🔸 TRANSFORM - Mengubah format teks                         ║
║     TRANSFORM uppercase(kolom)                               ║
║     TRANSFORM lowercase(kolom)                               ║
║     TRANSFORM capitalize(kolom)                              ║
║     TRANSFORM length(kolo

## Langkah 7: Coba Query! 🎉

### Test 1: Cari HP Samsung

In [13]:
# TEST CASE 1: Filter berdasarkan merek
print("📱 TEST CASE 1: Filter HP Samsung")
print("=" * 60)

result = engine.query("FILTER merek = 'Samsung'", verbose=True)

print("\n📊 HASIL:")
result[['nama', 'merek', 'harga', 'ram']]

📱 TEST CASE 1: Filter HP Samsung
🔍 QUERY: FILTER merek = 'Samsung'

📌 STEP 1: Lexical Analysis (Tokenisasi)
   Tokens: ["Token(KEYWORD, 'FILTER', pos=0)", "Token(IDENTIFIER, 'merek', pos=7)", "Token(OPERATOR, '=', pos=13)", "Token(STRING, 'Samsung', pos=15)"]

📌 STEP 2: Syntax Analysis (Parsing)
   AST Root: filter
   Children: 1 node(s)
📍 Riwayat Transisi State:
   START →    FILTER_STATE →    CONDITION →    ACCEPT

📌 STEP 3: Execution (Interpretasi)
   Hasil: DataFrame dengan 2 baris

📊 HASIL:


,nama,merek,harga,ram
0,Galaxy S24 Ultra,Samsung,21999000,12
10,Samsung Galaxy A54,Samsung,5499000,8


### Test 2: Cari HP Murah (dibawah 5 juta)

In [14]:
# TEST CASE 2: Filter berdasarkan harga
print("💰 TEST CASE 2: Filter HP Murah (< 5 juta)")
print("=" * 60)

result = engine.query("FILTER harga < 5000000", verbose=True)

print("\n📊 HASIL:")
result[['nama', 'merek', 'harga', 'ram', 'storage']]

💰 TEST CASE 2: Filter HP Murah (< 5 juta)
🔍 QUERY: FILTER harga < 5000000

📌 STEP 1: Lexical Analysis (Tokenisasi)
   Tokens: ["Token(KEYWORD, 'FILTER', pos=0)", "Token(IDENTIFIER, 'harga', pos=7)", "Token(OPERATOR, '<', pos=13)", "Token(NUMBER, '5000000', pos=15)"]

📌 STEP 2: Syntax Analysis (Parsing)
   AST Root: filter
   Children: 1 node(s)
📍 Riwayat Transisi State:
   START →    FILTER_STATE →    CONDITION →    ACCEPT

📌 STEP 3: Execution (Interpretasi)
   Hasil: DataFrame dengan 4 baris

📊 HASIL:


,nama,merek,harga,ram,storage
2,Redmi Note 13 Pro,Xiaomi,3999000,8,128
12,Redmi 13C,Xiaomi,1799000,4,128
13,Infinix Note 30 Pro,Infinix,3299000,8,256
14,Tecno Spark 20 Pro,Tecno,2499000,8,256


### Test 3: Ubah Nama Jadi HURUF BESAR

In [15]:
# TEST CASE 3: Transformasi teks
print("🔠 TEST CASE 3: Transform ke Uppercase")
print("=" * 60)

result = engine.query("TRANSFORM uppercase(nama)", verbose=True)

print("\n📊 HASIL (5 data pertama):")
result[['nama', 'uppercase_nama', 'merek']].head()

🔠 TEST CASE 3: Transform ke Uppercase
🔍 QUERY: TRANSFORM uppercase(nama)

📌 STEP 1: Lexical Analysis (Tokenisasi)
   Tokens: ["Token(KEYWORD, 'TRANSFORM', pos=0)", "Token(FUNCTION, 'uppercase', pos=10)", "Token(LPAREN, '(', pos=19)", "Token(IDENTIFIER, 'nama', pos=20)", "Token(RPAREN, ')', pos=24)"]

📌 STEP 2: Syntax Analysis (Parsing)
   AST Root: transform
   Children: 1 node(s)
📍 Riwayat Transisi State:
   START →    TRANSFORM_STATE →    FUNCTION_CALL →    ACCEPT

📌 STEP 3: Execution (Interpretasi)
   Hasil: DataFrame dengan 20 baris

📊 HASIL (5 data pertama):


,nama,uppercase_nama,merek
0,Galaxy S24 Ultra,GALAXY S24 ULTRA,Samsung
1,iPhone 15 Pro Max,IPHONE 15 PRO MAX,Apple
2,Redmi Note 13 Pro,REDMI NOTE 13 PRO,Xiaomi
3,ROG Phone 8 Pro,ROG PHONE 8 PRO,ASUS
4,Pixel 8 Pro,PIXEL 8 PRO,Google


### Test 4: Hitung Statistik Harga

In [16]:
# TEST CASE 4: Analisis statistik
print("📊 TEST CASE 4: Statistik Harga")
print("=" * 60)

result = engine.query("STATS count(nama), average(harga), min(harga), max(harga)", verbose=True)

print("\n📊 HASIL STATISTIK:")
for key, value in result.items():
    if 'harga' in key and key != 'count(nama)':
        print(f"   {key}: Rp {value:,.0f}")
    else:
        print(f"   {key}: {value}")

📊 TEST CASE 4: Statistik Harga
🔍 QUERY: STATS count(nama), average(harga), min(harga), max(harga)

📌 STEP 1: Lexical Analysis (Tokenisasi)
   Tokens: ["Token(KEYWORD, 'STATS', pos=0)", "Token(FUNCTION, 'count', pos=6)", "Token(LPAREN, '(', pos=11)", "Token(IDENTIFIER, 'nama', pos=12)", "Token(RPAREN, ')', pos=16)", "Token(COMMA, ',', pos=17)", "Token(FUNCTION, 'average', pos=19)", "Token(LPAREN, '(', pos=26)", "Token(IDENTIFIER, 'harga', pos=27)", "Token(RPAREN, ')', pos=32)", "Token(COMMA, ',', pos=33)", "Token(FUNCTION, 'min', pos=35)", "Token(LPAREN, '(', pos=38)", "Token(IDENTIFIER, 'harga', pos=39)", "Token(RPAREN, ')', pos=44)", "Token(COMMA, ',', pos=45)", "Token(FUNCTION, 'max', pos=47)", "Token(LPAREN, '(', pos=50)", "Token(IDENTIFIER, 'harga', pos=51)", "Token(RPAREN, ')', pos=56)"]

📌 STEP 2: Syntax Analysis (Parsing)
   AST Root: stats
   Children: 4 node(s)
📍 Riwayat Transisi State:
   START →    STATS_STATE →    FUNCTION_CALL →    STATS_STATE →    FUNCTION_CALL →    STATS

### Test 5: Cari HP Xiaomi dengan RAM Besar

In [17]:
# TEST CASE 5: Filter dengan kondisi ganda
print("🔗 TEST CASE 5: Filter Xiaomi dengan RAM >= 8GB")
print("=" * 60)

result = engine.query("FILTER merek = 'Xiaomi' AND ram >= 8", verbose=True)

print("\n📊 HASIL:")
result[['nama', 'merek', 'harga', 'ram', 'storage']]

🔗 TEST CASE 5: Filter Xiaomi dengan RAM >= 8GB
🔍 QUERY: FILTER merek = 'Xiaomi' AND ram >= 8

📌 STEP 1: Lexical Analysis (Tokenisasi)
   Tokens: ["Token(KEYWORD, 'FILTER', pos=0)", "Token(IDENTIFIER, 'merek', pos=7)", "Token(OPERATOR, '=', pos=13)", "Token(STRING, 'Xiaomi', pos=15)", "Token(KEYWORD, 'AND', pos=24)", "Token(IDENTIFIER, 'ram', pos=28)", "Token(OPERATOR, '>=', pos=32)", "Token(NUMBER, '8', pos=35)"]

📌 STEP 2: Syntax Analysis (Parsing)
   AST Root: filter
   Children: 2 node(s)
📍 Riwayat Transisi State:
   START →    FILTER_STATE →    CONDITION →    FILTER_STATE →    CONDITION →    ACCEPT

📌 STEP 3: Execution (Interpretasi)
   Hasil: DataFrame dengan 2 baris

📊 HASIL:


,nama,merek,harga,ram,storage
2,Redmi Note 13 Pro,Xiaomi,3999000,8,128
9,Poco F6 Pro,Xiaomi,7499000,12,256


### Ringkasan Semua Test

In [18]:
# RINGKASAN SEMUA TEST CASES
print("=" * 70)
print("📋 RINGKASAN HASIL SEMUA TEST CASES")
print("=" * 70)

test_cases = [
    ("Test 1", "FILTER merek = 'Samsung'", "Filter HP Samsung"),
    ("Test 2", "FILTER harga < 5000000", "Filter HP murah < 5 juta"),
    ("Test 3", "TRANSFORM uppercase(nama)", "Ubah nama jadi huruf besar"),
    ("Test 4", "STATS count(nama), average(harga)", "Hitung statistik"),
    ("Test 5", "FILTER merek = 'Xiaomi' AND ram >= 8", "Filter kombinasi AND"),
]

for name, query, description in test_cases:
    try:
        result = engine.query(query)
        if isinstance(result, pd.DataFrame):
            status = f"✅ {len(result)} data"
        else:
            status = f"✅ {result}"
    except Exception as e:
        status = f"❌ Error: {e}"
    
    print(f"\n{name}: {description}")
    print(f"   Query: {query}")
    print(f"   Hasil: {status}")

📋 RINGKASAN HASIL SEMUA TEST CASES

Test 1: Filter HP Samsung
   Query: FILTER merek = 'Samsung'
   Hasil: ✅ 2 data

Test 2: Filter HP murah < 5 juta
   Query: FILTER harga < 5000000
   Hasil: ✅ 4 data

Test 3: Ubah nama jadi huruf besar
   Query: TRANSFORM uppercase(nama)
   Hasil: ✅ 20 data

Test 4: Hitung statistik
   Query: STATS count(nama), average(harga)
   Hasil: ✅ {'count(nama)': 20, 'average(harga)': np.float64(12304000.0)}

Test 5: Filter kombinasi AND
   Query: FILTER merek = 'Xiaomi' AND ram >= 8
   Hasil: ✅ 2 data


## ✅ Kesimpulan

### Apa yang Sudah Dibuat:

| Bagian | Tugas | Status |
|--------|-------|--------|
| **Lexer** | Memecah perintah jadi kata-kata | ✅ Selesai |
| **Parser** | Memahami struktur perintah | ✅ Selesai |
| **Automata** | Cek perintah benar/salah | ✅ Selesai |
| **Interpreter** | Menjalankan perintah | ✅ Selesai |
| **Query Engine** | Menggabungkan semua | ✅ Selesai |

### Perintah yang Bisa Dipakai:

```
FILTER merek = 'Samsung'         → Cari HP Samsung
FILTER harga < 5000000           → Cari HP dibawah 5 juta  
TRANSFORM uppercase(nama)        → Ubah nama jadi HURUF BESAR
STATS average(harga)             → Hitung rata-rata harga
```

### Apa yang Dipelajari:

1. **Lexer** - Cara memecah teks jadi bagian-bagian kecil
2. **Parser** - Cara memahami struktur bahasa
3. **Automata** - Cara membuat mesin yang bisa validasi
4. **Interpreter** - Cara menjalankan perintah

---
**Semua 5 Test berhasil! ✅**

## 🎮 Mode Interaktif - Coba Sendiri!

Sekarang giliran kamu untuk mencoba! Jalankan cell dibawah dan ketik perintah sendiri.

### Contoh yang Bisa Dicoba:

| Perintah | Fungsi |
|----------|--------|
| `FILTER merek = 'Samsung'` | Cari HP Samsung |
| `FILTER harga < 10000000` | Cari HP dibawah 10 juta |
| `FILTER ram >= 8` | Cari HP dengan RAM 8GB keatas |
| `TRANSFORM uppercase(nama)` | Ubah nama jadi HURUF BESAR |
| `STATS average(harga)` | Hitung rata-rata harga |

Ketik `bantuan` untuk lihat panduan lengkap!

In [21]:
# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║                    🎮 MODE INTERAKTIF - MASUKKAN QUERY ANDA                  ║
# ╚══════════════════════════════════════════════════════════════════════════════╝

def tampilkan_bantuan():
    """Menampilkan panduan penggunaan query dalam Bahasa Indonesia"""
    print("""
╔══════════════════════════════════════════════════════════════════════════════╗
║                         📖 PANDUAN QUERY - BAHASA INDONESIA                  ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  🔍 FILTER (Menyaring Data)                                                  ║
║  ─────────────────────────────────────────────────────────────────────────── ║
║  • FILTER merek = 'Samsung'         → Tampilkan HP Samsung saja              ║
║  • FILTER harga < 5000000           → HP dengan harga dibawah 5 juta         ║
║  • FILTER harga >= 10000000         → HP dengan harga 10 juta keatas         ║
║  • FILTER ram = 8                   → HP dengan RAM 8GB                      ║
║  • FILTER merek = 'Xiaomi' AND ram >= 8  → Xiaomi dengan RAM 8GB+            ║
║  • FILTER harga < 5000000 OR ram >= 12   → Murah ATAU RAM besar              ║
║  • FILTER deskripsi CONTAINS 'gaming'    → Cari kata "gaming" di deskripsi   ║
║                                                                              ║
║  🔠 TRANSFORM (Mengubah Teks)                                                ║
║  ─────────────────────────────────────────────────────────────────────────── ║
║  • TRANSFORM uppercase(nama)        → Ubah nama jadi HURUF BESAR             ║
║  • TRANSFORM lowercase(merek)       → Ubah merek jadi huruf kecil            ║
║  • TRANSFORM capitalize(nama)       → Ubah Jadi Huruf Kapital Tiap Kata      ║
║  • TRANSFORM length(deskripsi)      → Hitung jumlah karakter deskripsi       ║
║                                                                              ║
║  📊 STATS (Statistik)                                                        ║
║  ─────────────────────────────────────────────────────────────────────────── ║
║  • STATS count(nama)                → Hitung jumlah HP                       ║
║  • STATS average(harga)             → Rata-rata harga                        ║
║  • STATS min(harga), max(harga)     → Harga termurah & termahal              ║
║  • STATS sum(baterai)               → Total kapasitas baterai                ║
║  • STATS distinct(merek)            → Jumlah merek berbeda                   ║
║                                                                              ║
║  📋 KOLOM YANG TERSEDIA:                                                     ║
║  nama, merek, harga, ram, storage, baterai, kamera, layar,                   ║
║  sistem_operasi, deskripsi                                                   ║
║                                                                              ║
║  💡 Ketik 'help' untuk bantuan, 'contoh' untuk contoh, 'quit' untuk keluar   ║
╚══════════════════════════════════════════════════════════════════════════════╝
    """)

def tampilkan_contoh():
    """Menampilkan contoh-contoh query"""
    print("""
╔══════════════════════════════════════════════════════════════════════════════╗
║                           📝 CONTOH QUERY                                    ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  1️⃣  FILTER merek = 'Samsung'                                                ║
║      → Menampilkan semua HP Samsung                                          ║
║                                                                              ║
║  2️⃣  FILTER harga < 5000000                                                  ║
║      → Menampilkan HP dengan harga dibawah 5 juta                            ║
║                                                                              ║
║  3️⃣  TRANSFORM uppercase(nama)                                               ║
║      → Mengubah nama HP menjadi HURUF BESAR                                  ║
║                                                                              ║
║  4️⃣  STATS count(nama), average(harga)                                       ║
║      → Menghitung jumlah HP dan rata-rata harga                              ║
║                                                                              ║
║  5️⃣  FILTER merek = 'Xiaomi' AND ram >= 8                                    ║
║      → Menampilkan HP Xiaomi dengan RAM 8GB atau lebih                       ║
║                                                                              ║
╚══════════════════════════════════════════════════════════════════════════════╝
    """)

def format_harga(nilai):
    """Format angka menjadi format Rupiah"""
    return f"Rp {nilai:,.0f}".replace(",", ".")

def tampilkan_hasil(hasil, query_type):
    """Menampilkan hasil query dengan format yang bagus"""
    print("\n" + "═" * 70)
    print("📊 HASIL QUERY")
    print("═" * 70)
    
    if isinstance(hasil, pd.DataFrame):
        print(f"✅ Ditemukan {len(hasil)} data\n")
        
        # Tampilkan dengan format yang lebih bagus
        if len(hasil) > 0:
            # Pilih kolom yang akan ditampilkan berdasarkan query
            if 'uppercase_' in str(hasil.columns.tolist()) or 'lowercase_' in str(hasil.columns.tolist()) or 'length_' in str(hasil.columns.tolist()):
                # Untuk transform, tampilkan kolom asli dan hasil transform
                display(hasil)
            else:
                # Untuk filter, tampilkan kolom penting
                cols_to_show = ['nama', 'merek', 'harga', 'ram', 'storage']
                cols_available = [c for c in cols_to_show if c in hasil.columns]
                display(hasil[cols_available])
                
                # Tampilkan statistik singkat
                if 'harga' in hasil.columns:
                    print(f"\n📈 Statistik Harga:")
                    print(f"   • Termurah  : {format_harga(hasil['harga'].min())}")
                    print(f"   • Termahal  : {format_harga(hasil['harga'].max())}")
                    print(f"   • Rata-rata : {format_harga(hasil['harga'].mean())}")
        else:
            print("⚠️ Tidak ada data yang cocok dengan kriteria")
    
    elif isinstance(hasil, dict):
        print("✅ Hasil Statistik:\n")
        for key, value in hasil.items():
            if 'harga' in key.lower() and isinstance(value, (int, float)):
                print(f"   📌 {key}: {format_harga(value)}")
            else:
                print(f"   📌 {key}: {value}")
    
    print("\n" + "═" * 70)

def jalankan_query_interaktif():
    """Fungsi utama untuk mode interaktif"""
    print("""
╔══════════════════════════════════════════════════════════════════════════════╗
║          🚀 SELAMAT DATANG DI SMARTPHONE QUERY ENGINE 🚀                     ║
║                                                                              ║
║  Sistem ini membantu Anda mencari dan menganalisis data smartphone           ║
║  dengan mudah menggunakan bahasa query sederhana.                            ║
║                                                                              ║
║  💡 Ketik 'help' untuk melihat panduan lengkap                               ║
║  💡 Ketik 'contoh' untuk melihat contoh query                                ║
║  💡 Ketik 'data' untuk melihat semua data                                    ║
║  💡 Ketik 'quit' atau 'keluar' untuk berhenti                                ║
╚══════════════════════════════════════════════════════════════════════════════╝
    """)
    
    nomor_query = 1
    
    while True:
        print(f"\n{'─' * 70}")
        try:
            query_input = input(f"📝 Query #{nomor_query} (ketik 'help' untuk bantuan): ").strip()
        except EOFError:
            print("\n👋 Terima kasih telah menggunakan Smartphone Query Engine!")
            break
        
        # Cek perintah khusus
        if query_input.lower() in ['quit', 'exit', 'keluar', 'q']:
            print("\n👋 Terima kasih telah menggunakan Smartphone Query Engine!")
            print("   Sampai jumpa lagi! 🎉")
            break
        
        elif query_input.lower() in ['help', 'bantuan', 'h', '?']:
            tampilkan_bantuan()
            continue
        
        elif query_input.lower() in ['contoh', 'example', 'examples']:
            tampilkan_contoh()
            continue
        
        elif query_input.lower() in ['data', 'show', 'lihat']:
            print("\n📱 SEMUA DATA SMARTPHONE:")
            print("═" * 70)
            display(df[['nama', 'merek', 'harga', 'ram', 'storage']])
            continue
        
        elif query_input.lower() in ['kolom', 'columns']:
            print("\n📋 KOLOM YANG TERSEDIA:")
            for col in df.columns:
                print(f"   • {col}")
            continue
        
        elif query_input == '':
            print("⚠️ Query kosong! Silakan masukkan query atau ketik 'help' untuk bantuan.")
            continue
        
        # Jalankan query
        print(f"\n🔄 Menjalankan: {query_input}")
        hasil = engine.query(query_input, verbose=False)
        
        if hasil is not None:
            tampilkan_hasil(hasil, query_input)
            nomor_query += 1
        else:
            print("\n❌ Query gagal! Periksa kembali format query Anda.")
            print("   Ketik 'help' untuk melihat panduan atau 'contoh' untuk contoh query.")

# Jalankan mode interaktif
print("🎮 Menjalankan Mode Interaktif...")
print("   (Jalankan cell ini untuk mulai memasukkan query Anda sendiri)\n")
jalankan_query_interaktif()

🎮 Menjalankan Mode Interaktif...
   (Jalankan cell ini untuk mulai memasukkan query Anda sendiri)


╔══════════════════════════════════════════════════════════════════════════════╗
║          🚀 SELAMAT DATANG DI SMARTPHONE QUERY ENGINE 🚀                     ║
║                                                                              ║
║  Sistem ini membantu Anda mencari dan menganalisis data smartphone           ║
║  dengan mudah menggunakan bahasa query sederhana.                            ║
║                                                                              ║
║  💡 Ketik 'help' untuk melihat panduan lengkap                               ║
║  💡 Ketik 'contoh' untuk melihat contoh query                                ║
║  💡 Ketik 'data' untuk melihat semua data                                    ║
║  💡 Ketik 'quit' atau 'keluar' untuk berhenti                                ║
╚══════════════════════════════════════════════════════════════════════════════╝
    

─────────

,nama,merek,harga,ram,storage
0,Galaxy S24 Ultra,Samsung,21999000,12,256
10,Samsung Galaxy A54,Samsung,5499000,8,128



📈 Statistik Harga:
   • Termurah  : Rp 5.499.000
   • Termahal  : Rp 21.999.000
   • Rata-rata : Rp 13.749.000

══════════════════════════════════════════════════════════════════════

──────────────────────────────────────────────────────────────────────

👋 Terima kasih telah menggunakan Smartphone Query Engine!
   Sampai jumpa lagi! 🎉


### Jalankan Mode Interaktif

Jalankan cell ini, lalu ketik perintah kamu!

In [25]:
# ╔══════════════════════════════════════════════════════════════════════════════╗
# ║  📝 MASUKKAN QUERY ANDA DI SINI - GANTI NILAI query_saya                     ║
# ╚══════════════════════════════════════════════════════════════════════════════╝

# ═══════════════════════════════════════════════════════════════════════════════
# 🔽🔽🔽 GANTI QUERY DI BAWAH INI DENGAN QUERY YANG ANDA INGINKAN 🔽🔽🔽
# ═══════════════════════════════════════════════════════════════════════════════

query_saya = "FILTER merek = 'Samsung' AND harga < 10000000"
# ═══════════════════════════════════════════════════════════════════════════════
# 🔼🔼🔼 GANTI QUERY DI ATAS, LALU JALANKAN CELL INI (Shift+Enter) 🔼🔼🔼
# ═══════════════════════════════════════════════════════════════════════════════

# Contoh query lain yang bisa Anda coba (hapus # di depan untuk mengaktifkan):
# query_saya = "FILTER harga < 5000000"
# query_saya = "FILTER merek = 'Xiaomi' AND ram >= 8"
# query_saya = "FILTER deskripsi CONTAINS 'gaming'"
# query_saya = "TRANSFORM uppercase(nama)"
# query_saya = "STATS count(nama), average(harga), min(harga), max(harga)"

print("╔" + "═" * 68 + "╗")
print("║" + " 🔍 MENJALANKAN QUERY ".center(68) + "║")
print("╚" + "═" * 68 + "╝")
print(f"\n📝 Query: {query_saya}\n")

# Jalankan query
hasil_query = engine.query(query_saya, verbose=True)

# Tampilkan hasil dengan format bagus
if hasil_query is not None:
    print("\n" + "═" * 70)
    print("📊 HASIL QUERY")
    print("═" * 70)
    
    if isinstance(hasil_query, pd.DataFrame):
        print(f"✅ Ditemukan {len(hasil_query)} data\n")
        display(hasil_query)
        
        # Statistik tambahan untuk filter
        if 'harga' in hasil_query.columns and len(hasil_query) > 0:
            print(f"\n💰 Statistik Harga dari Hasil:")
            print(f"   • Termurah  : Rp {hasil_query['harga'].min():,.0f}".replace(",", "."))
            print(f"   • Termahal  : Rp {hasil_query['harga'].max():,.0f}".replace(",", "."))
            print(f"   • Rata-rata : Rp {hasil_query['harga'].mean():,.0f}".replace(",", "."))
    else:
        print("✅ Hasil Statistik:\n")
        for k, v in hasil_query.items():
            if 'harga' in k.lower() and isinstance(v, (int, float)):
                print(f"   📌 {k}: Rp {float(v):,.0f}".replace(",", "."))
            else:
                print(f"   📌 {k}: {v}")
    
    print("\n" + "═" * 70)
else:
    print("\n❌ Query gagal! Periksa format query Anda.")

╔════════════════════════════════════════════════════════════════════╗
║                        🔍 MENJALANKAN QUERY                         ║
╚════════════════════════════════════════════════════════════════════╝

📝 Query: FILTER merek = 'Samsung' AND harga < 10000000

🔍 QUERY: FILTER merek = 'Samsung' AND harga < 10000000

📌 STEP 1: Lexical Analysis (Tokenisasi)
   Tokens: ["Token(KEYWORD, 'FILTER', pos=0)", "Token(IDENTIFIER, 'merek', pos=7)", "Token(OPERATOR, '=', pos=13)", "Token(STRING, 'Samsung', pos=15)", "Token(KEYWORD, 'AND', pos=25)", "Token(IDENTIFIER, 'harga', pos=29)", "Token(OPERATOR, '<', pos=35)", "Token(NUMBER, '10000000', pos=37)"]

📌 STEP 2: Syntax Analysis (Parsing)
   AST Root: filter
   Children: 2 node(s)
📍 Riwayat Transisi State:
   START →    FILTER_STATE →    CONDITION →    FILTER_STATE →    CONDITION →    ACCEPT

📌 STEP 3: Execution (Interpretasi)
   Hasil: DataFrame dengan 1 baris

══════════════════════════════════════════════════════════════════════
📊 HASI

,id,nama,merek,harga,ram,storage,baterai,kamera,layar,sistem_operasi,deskripsi
10,11,Samsung Galaxy A54,Samsung,5499000,8,128,5000,50,6.4,Android 13,Mid-range Samsung dengan fitur premium dan upd...



💰 Statistik Harga dari Hasil:
   • Termurah  : Rp 5.499.000
   • Termahal  : Rp 5.499.000
   • Rata-rata : Rp 5.499.000

══════════════════════════════════════════════════════════════════════
